<div class="alert alert-info">
    <center><b>Set up Notebook</b></center>
</div>

In [1]:
%pip install pandas tqdm pathlib 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from utils.postgres import initialize_db
from utils.git import download

Added finer to sys.path


<div class="alert alert-info">
    <center><b>Create schema, database and tables</b></center>
</div>

In [4]:
initialize_db()

<div class="alert alert-info">
    <center><b>Get repositories data from csv file</b></center>
</div>

In [5]:
repos = pd.read_csv('../code_samples.csv', skiprows=1)
repos = repos.dropna(subset=['html_url'])

<div class="alert alert-info">
    <center><b>Cloning repositories in a subfolder</b></center>
</div>

In [6]:
for i in tqdm(range(len(repos)), desc=f"Downloading Repositories"):
    repo = repos.iloc[i]
    repo_ecosystem = repo['html_url'].split('/')[-2]
    repo_name = repo['name']
    sample_name = f"{repo_ecosystem}/{repo_name}"
    download(sample_name)

Repository Azure-Samples/acr-dotnet-manage-azure-container-registry already downloaded
Repository Azure-Samples/acr-dotnet-manage-azure-container-registry-with-webhooks already downloaded
Repository Azure-Samples/acr-tasks already downloaded
Repository Azure-Samples/active-directory-b2c-dotnet-desktop already downloaded
Repository Azure-Samples/active-directory-b2c-javascript-nodejs-webapi already downloaded
Repository Azure-Samples/active-directory-b2c-xamarin-native already downloaded
Repository Azure-Samples/active-directory-dotnet-native-aspnetcore-v2 already downloaded
Repository Azure-Samples/active-directory-dotnetcore-console-up-v2 already downloaded
Repository Azure-Samples/active-directory-verifiable-credentials-node already downloaded
Repository Azure-Samples/agent-openai-python-prompty already downloaded
Repository Azure-Samples/ai-rag-chat-evaluator already downloaded
Repository Azure-Samples/aistudio-python-promptflow-sample already downloaded
Repository Azure-Samples/ais